# Preprocessing

For now we need just pandas for parsing dataset and transformers for BERT

In [1]:
import pandas as pd
from transformers import *
import numpy as np
import torch

Read excel file, skip headers and 1st row with column nasmes

In [2]:
df= pd.read_excel('../input/dataset.xlsx', header=None, skiprows=1)

For our smaller version of dataset just take two features that we are interested in, namely T PL and GOLD<T,H>

In [3]:
dataset = df[[4, 5]]

In [4]:
dataset.head

<bound method NDFrame.head of                                                       4  5
0     Ja na to, że mam klasówkę, że zawalę biologię ...  N
1            Ani że na progu stanął właśnie Romanyczko.  ?
2                  Potem, że się przywiezie Chińczyków.  N
3     Słabł coraz bardziej, czuł, że to kwestia zale...  N
4     Jednak ludzie bardzo ciężko chorzy, gdy już cz...  N
...                                                 ... ..
2591  Udało się ustalić, że obie dziewczynki mogą pr...  N
2592  Ustalono, że łączą go z niektórymi członkami g...  N
2593  Tu będą mogli zobaczyć, że Państwo Izrael ma k...  E
2594  Oczywiście, że można kochać, jeśli zobaczy się...  N
2595  Oczywiście, że można kochać, jeśli zobaczy się...  N

[2596 rows x 2 columns]>

Change labels of ECN for numeric values

In [5]:
labels = {'E':0,'C':1,'N':2,'?':3}

for idx, row in dataset.iterrows():
    row[5] = labels[row[5]]

In [6]:
dataset

,4,5
0,"Ja na to, że mam klasówkę, że zawalę biologię ...",2
1,Ani że na progu stanął właśnie Romanyczko.,3
2,"Potem, że się przywiezie Chińczyków.",2
3,"Słabł coraz bardziej, czuł, że to kwestia zale...",2
4,"Jednak ludzie bardzo ciężko chorzy, gdy już cz...",2
...,...,...
2591,"Udało się ustalić, że obie dziewczynki mogą pr...",2
2592,"Ustalono, że łączą go z niektórymi członkami g...",2
2593,"Tu będą mogli zobaczyć, że Państwo Izrael ma k...",0
2594,"Oczywiście, że można kochać, jeśli zobaczy się...",2


In [7]:
dataset[5].value_counts()

2    1614
0     830
3      88
1      64
Name: 5, dtype: int64

In [8]:
max_len = 0
m_idx = 0
for idx,row in dataset.iterrows():
    l = len(row[4])
    if l > max_len:
        max_len = l
        m_idx = idx
print(m_idx)

360


In [9]:
print(dataset[4][m_idx])

Ci z koalicji rządzącej, ci posłowie, którzy w trakcie posiedzenia sejmowej Komisji Polityki Gospodarczej, Budżetu i Finansów z uporem, myślę, godnym lepszej sprawy, bronili faktu, aby rząd łącznie z budżetem - zapowiadam, łącznie z budżetem, bo mamy świadomość, że nie było i nie ma czasu na to, żeby wcześniej rozpatrywać założenia, a potem przedkładać budżet - przedłożył założenia polityki społeczno-gospodarczej, jako dokumentu ujmującego tę politykę w dłuższym czasie, posłowie, którzy nie dopuścili do takiego rozstrzygnięcia - przypominam, tylko jednym głosem - źle służą również rządowi.


In [10]:
!pip install sklearn
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 7.1 MB 917 kB/s eta 0:00:01|████████                        | 1.8 MB 917 kB/s eta 0:00:06
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=c7cf73edeb5f3ab9eb5e452ce71d0db6bce9bae51bf6ffd79148485dd2534fc1
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


# Polbert

Here we import pretrained version of Bert for Polish language

In [11]:
model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

## Tokenization

Tokenize every sentence from our dataset

In [12]:
tokenized = dataset[4].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [13]:
tokenized.values

array([list([2, 2055, 1898, 1907, 16, 2095, 2040, 15701, 1889, 3118, 16, 2095, 32091, 1014, 12071, 1014, 51, 1893, 15887, 1012, 1906, 27508, 2165, 4785, 18, 4]),
       list([2, 2937, 2095, 1898, 26468, 40126, 1019, 29480, 6977, 1013, 7150, 18, 4]),
       list([2, 3006, 16, 2095, 2243, 11319, 2713, 11007, 8572, 1889, 18, 4]),
       ...,
       list([2, 2030, 40221, 4949, 38584, 16, 2095, 2013, 2803, 12250, 1994, 26825, 2437, 48721, 16, 2030, 14229, 2114, 2808, 21694, 40575, 4955, 52867, 9417, 16, 2030, 8822, 1948, 2243, 24391, 3117, 4955, 52867, 1014, 27972, 16, 2030, 48570, 41886, 2243, 51, 5245, 50930, 36755, 50719, 15956, 23652, 2054, 18, 4]),
       list([2, 31040, 16, 2095, 29801, 6289, 1015, 16, 10865, 3403, 2243, 16, 2095, 2259, 16879, 1015, 1893, 40065, 1923, 2243, 68, 4109, 16, 2095, 2499, 3089, 1906, 3115, 3410, 18, 4]),
       list([2, 31040, 16, 2095, 29801, 6289, 1015, 16, 10865, 3403, 2243, 16, 2095, 2259, 16879, 1015, 1893, 40065, 1923, 2243, 68, 4109, 16, 2095, 2499, 

## Padding

We need to have input of the same lenght in order to feed BERT once (we will get performance speed). This will be some simple python string manipulations

In [14]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [15]:
max_len

146

Maybe we can consider shortening some sentences here

In [16]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [17]:
padded

array([[    2,  2055,  1898, ...,     0,     0,     0],
       [    2,  2937,  2095, ...,     0,     0,     0],
       [    2,  3006,    16, ...,     0,     0,     0],
       ...,
       [    2,  2030, 40221, ...,     0,     0,     0],
       [    2, 31040,    16, ...,     0,     0,     0],
       [    2, 31040,    16, ...,     0,     0,     0]])

In [18]:
np.array(padded).shape

(2596, 146)

In [19]:
input_ids = torch.tensor(np.array(padded))

In [20]:
input_ids

tensor([[    2,  2055,  1898,  ...,     0,     0,     0],
        [    2,  2937,  2095,  ...,     0,     0,     0],
        [    2,  3006,    16,  ...,     0,     0,     0],
        ...,
        [    2,  2030, 40221,  ...,     0,     0,     0],
        [    2, 31040,    16,  ...,     0,     0,     0],
        [    2, 31040,    16,  ...,     0,     0,     0]])

In [ ]:
with torch.no_grad():
    last_hidden_states = model(input_ids)

In [1]:
!jupyter kernelspec list

Available kernels:
  python3    /usr/local/share/jupyter/kernels/python3
